In [ ]:
import pandas as pd
import os

path = './MAMe_metadata/MAMe_labels.csv'

In [ ]:
dataframe = pd.read_csv(path, header=None)

In [ ]:
dict(zip(dataframe[1], dataframe[0]))

In [ ]:
dataframe.set_index(1)[0].to_dict()

In [ ]:
image_names = dataframe['Image file'].to_numpy()

In [ ]:
import cv2

images_path = os.path.join('./', 'MAMe_data_256', 'data_256')

are_all_256 = True

for image_name in image_names:
    image_path = os.path.join(images_path, image_name)
    image = cv2.imread(image_path)

    if image is None:
        print(image_name)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    if image.shape != (256, 256, 3):
        are_all_256 = False


print(are_all_256)

In [ ]:
import logging
from PIL import Image
from PIL import UnidentifiedImageError


for image_name in image_names:
    image_path = os.path.join(images_path, image_name)

    try:
        img = Image.open(image_path)
    
    except UnidentifiedImageError as e:
        logging.warning(e)
        logging.warning(f'Detected malformatted image: {image_name}. Skipping.')
        continue
    
    if img.mode == 'RGBA':
        logging.warning(f'Detected RGBA: {image_name}. Converting into RGB.')
        
    
    if img.mode == 'P':
        logging.warning(f'Detected P: {image_name}. Converting into RGB.')
       
    
    if img.mode == 'L':
        logging.warning(f'Detected BW image: {image_name}')


In [ ]:
img.show()

In [ ]:
'456.png Yep'.strip().split()

In [3]:
from tensorflow import keras
import tensorflow as tf

model = keras.Sequential()
model.add(keras.layers.Conv2D(8, (3, 3), activation='relu', input_shape=(256,256,3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(16, 3, 3, activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(8, activation='relu'))
model.add(keras.layers.Dense(10, activation=(tf.nn.softmax)))

In [4]:
model_json = model.to_json()

In [12]:
import json
with open('model.json', 'w') as json_file:
    json.dump(json.loads(model_json), json_file, indent=4)

In [4]:
import numpy as np
result = np.unique([1,2,3,1,3,1,4,5,1], return_counts=True)
result

(array([1, 2, 3, 4, 5]),
 array([0, 1, 2, 6, 7], dtype=int64),
 array([4, 1, 2, 1, 1], dtype=int64))

In [43]:
from models.basic_cnn import CNNModelWrapper

input_shape = (256, 256, 3)
kernel_size = 3
filters_list = [16, 32, 64]
use_batch_norm = True
dropout_percentage = 0
use_max_pooling = True
dense_units_list = [128, 256]
num_classes = 29

model_wrapper = CNNModelWrapper(input_shape, kernel_size, filters_list, dense_units_list, use_batch_norm, dropout_percentage, use_max_pooling, num_classes)
model_wrapper.save_summary('./')

x = tf.constant(1.0, shape = (32, *input_shape), dtype=tf.float32)
y = model_wrapper.model(x)
print(y.shape)

(32, 29)


In [1]:
import numpy as np
from utils.file_io_utils import *
from utils.image_utils import *

mean_std = load_mean_std_from_npy(os.path.join('./', 'processed_dataset', 'train_mean_std.npy'))
train_names = read_file_to_array(os.path.join('./', 'processed_dataset', 'dataset_splits', 'train_data.txt'))
data_path = os.path.join('./', 'MAMe_data_256', 'data_256')

image = load_image(data_path, train_names[1])
image = image_to_float(image)

print(np.mean(image, axis=(0, 1)))
print(np.std(image, axis=(0, 1)))

print(mean_std[0], mean_std[1])


norm = (image - mean_std[0]) / mean_std[1]

print(np.mean(norm, axis=(0, 1)))
print(np.std(norm, axis=(0, 1)))

[0.32905766 0.30947202 0.29506493]
[0.18233408 0.15392433 0.11436046]
[0.6191213  0.5868106  0.54119109] [0.26380283 0.26261466 0.26413966]
[-1.09927721 -1.05586115 -0.93166705]
[0.6911913  0.58612051 0.43296888]


In [69]:
print(np.mean(image, axis=(0, 1)))
print(np.std(image, axis=(0, 1)))

[0.27335929 0.22962311 0.15042509]
[0.22910468 0.18631586 0.14312487]


In [3]:
import tensorflow as tf

def augment_image(image):
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.5, 2.0)
    image = tf.image.random_saturation(image, 0.75, 1.25)
    image = tf.image.random_hue(image, 0.1)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image


image_augm = augment_image(image)


In [5]:
type(image_augm.numpy())

numpy.ndarray